# 4. 키워드 기반 필수 법령 조항 매핑 RAG 시스템
질문에서 핵심 키워드를 자동으로 감지하고, 사전에 정의된 키워드별 필수 조항을 검색하여 정확한 답변 생성하고자 합니다.

**핵심 아이디어**
- 주요 법률 키워드(소득세, 증여세 등)마다 필수 조항들을 미리 매핑
- 질문 분석 → 키워드 자동 감지 → 해당 키워드의 필수 조항 자동 검색
- 필수 조항 기반 검색으로 답변 정확도 향상

## Step 1: 환경 설정

In [47]:
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

llm = ChatUpstage()
embedding = UpstageEmbeddings(model='solar-embedding-1-large')
database = Chroma(
    collection_name='chroma-tax', 
    persist_directory="./chroma", 
    embedding_function=embedding
)

## Step 2: 키워드별 필수 조항 사전 정의

법률 키워드별 필수 조항들을 사전에 매핑해둡니다.

In [48]:
# 키워드별 필수 조항 매핑 사전
KEYWORD_ARTICLE_MAPPING = {
    "소득세": {
        "description": "근로소득 관련 세금 계산",
        "essential_articles": {
            "제47조": "근로소득공제",
            "제50조": "기본공제",
            "제51조의3": "연금보험료공제",
            "제52조": "특별소득공제 건강보험 고용보험",
            "제55조": "세율",
            "제59조": "근로소득세액공제"
        }
    },
    "증여세": {
        "description": "증여 관련 세금 계산",
        "essential_articles": {
            "제53조": "증여세 과세표준",
            "제54조": "증여세 세율",
            "제57조": "증여재산공제"
        }
    }
}

## Step 3: 질문에서 키워드 자동 감지

LLM을 사용해 질문에서 법률 키워드를 자동으로 감지하고,
사전에 정의된 키워드의 필수 조항들을 가져옵니다.

In [49]:
def detect_keyword_and_get_articles(question, keyword_mapping):
    # 등록된 키워드 목록
    registered_keywords = list(keyword_mapping.keys())
    
    # LLM으로 키워드 감지
    keyword_prompt = ChatPromptTemplate.from_template("""
다음 질문에서 법률/세금 관련 핵심 키워드를 찾으세요.
아래 등록된 키워드 중에서만 선택하세요:
{registered_keywords}

질문: {question}

발견된 키워드 1개만 출력하세요.
등록된 키워드가 없으면 '없음'이라고 출력하세요:
""")
    
    keyword_chain = keyword_prompt | llm | StrOutputParser()
    detected = keyword_chain.invoke({
        "question": question,
        "registered_keywords": ", ".join(registered_keywords)
    }).strip()
    
    print(f"감지된 키워드: {detected}")
    
    # 키워드가 사전에 있는지 확인
    if detected in keyword_mapping:
        essential_articles = keyword_mapping[detected]['essential_articles']
        return {
            'detected_keyword': detected,
            'essential_articles': essential_articles
        }
    else:
        return {
            'detected_keyword': None,
            'essential_articles': {}
        }

# 테스트
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
print(f"질문: {query}")

result = detect_keyword_and_get_articles(query, KEYWORD_ARTICLE_MAPPING)

질문: 연봉 5천만원인 직장인의 소득세는 얼마인가요?
감지된 키워드: 소득세


## Step 4: 프롬프트 정의

계산 예시와 단계별 명확한 지침을 포함한 전문적인 프롬프트

In [50]:
from langchain.prompts import PromptTemplate

income_tax_prompt = PromptTemplate(
    input_variables=["context", "question"], 
    template="""
당신은 세무 전문가입니다. 제공된 법령 문서를 정확히 적용하여 소득세를 계산하세요.

【참고 법령】
{context}

【질문】
{question}

【계산 지침】
1. 근로소득공제 (제47조) 적용
2. 과세표준 계산 (기본공제, 국민연금, 건강보험 등)
3. 산출세액 계산 (제55조 세율표 적용)
4. 근로소득세액공제 (제59조) 적용
5. 지방소득세 계산 (10%)

각 단계마다 적용한 법령 조항을 명시하고,
계산 과정을 단계별로 명확하게 보여주세요.
최종 답변은 간결하게 요약하세요.
"""
)

## Step 5: 커스텀 Retriever 생성

키워드 기반으로 필수 조항을 자동 검색하는 Retriever
동작 방식은 다음과 같습니다.
1. 질문에서 키워드 자동 감지
2. 사전에 정의된 필수 조항들 검색 (각 조항당 1개 문서)
3. 질문으로 추가 문서 검색 (k=3)
4. 중복 제거 후 반환

In [51]:
from langchain.schema import BaseRetriever, Document
from typing import List
from pydantic import ConfigDict

class KeywordBasedRetriever(BaseRetriever):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    database: Chroma
    keyword_mapping: dict
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        # 1. 키워드 감지 및 필수 조항 가져오기
        result = detect_keyword_and_get_articles(query, self.keyword_mapping)
        essential_articles = result['essential_articles']
        
        all_docs = []
        
        # 2. 필수 조항 검색
        if essential_articles:
            for article, description in essential_articles.items():
                search_query = f"{article} {description}"
                docs = self.database.similarity_search(search_query, k=1)
                if docs:
                    all_docs.extend(docs)
        
        # 3. 원본 질문으로 추가 검색
        additional_docs = self.database.similarity_search(query, k=3)
        all_docs.extend(additional_docs)
        
        # 4. 중복 제거
        seen_contents = set()
        unique_docs = []
        
        for doc in all_docs:
            content_preview = doc.page_content[:100]
            if content_preview not in seen_contents:
                seen_contents.add(content_preview)
                unique_docs.append(doc)
        
        return unique_docs
    
    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        return self._get_relevant_documents(query)

# Retriever 생성
custom_retriever = KeywordBasedRetriever(
    database=database,
    keyword_mapping=KEYWORD_ARTICLE_MAPPING
)

## Step 6: 최종 RAG Chain 구성

In [52]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    chain_type_kwargs={"prompt": income_tax_prompt},
    return_source_documents=True
)

## Step 7: 실행 및 결과 확인

In [53]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

print("💬 질문:", query)

result = qa_chain.invoke({"query": query})

print("💡 답변:")
print(result['result'])

💬 질문: 연봉 5천만원인 직장인의 소득세는 얼마인가요?
감지된 키워드: 소득세
💡 답변:
연봉 5천만원 직장인의 소득세를 계산해 보겠습니다.

**1. 근로소득공제 적용**

제47조(근로소득공제) ①에 따라 연봉 5천만원에 대한 근로소득공제를 계산합니다.

- 근로소득공제 = min(2천만원, 5천만원) = 2천만원

**2. 과세표준 계산**

근로소득공제 후 과세표준을 계산합니다. 기본공제(1,500만원)와 표준세액공제(13만원)를 적용합니다.

- 과세표준 = 5천만원 - 2천만원 - 1,500만원 - 13만원 = 2,487만원

**3. 산출세액 계산**

제55조(세율) ①에 따라 과세표준 2,487만원에 대한 산출세액을 계산합니다.

- 산출세액 = 84만원 + (1,400만원을 초과하는 금액의 15% * (2,487만원 - 1,400만원)) = 84만원 + 243.05만원 = 327.05만원

**4. 근로소득세액공제 적용**

제59조(근로소득세액공제) ①에 따라 근로소득세액공제를 계산합니다.

- 근로소득세액공제 = min(74만원, 327.05만원) = 74만원

- 결정세액 = 산출세액 - 근로소득세액공제 = 327.05만원 - 74만원 = 253.05만원

**5. 지방소득세 계산**

결정세액의 10%를 지방소득세로 계산합니다.

- 지방소득세 = 결정세액 * 10% = 253.05만원 * 10% = 25.305만원

**최종 결과**

연봉 5천만원 직장인의 소득세와 지방소득세를 합한 총 납부세액은 278.355만원입니다.
